## Real time Mask detection 

In [1]:
import os
import cv2
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split 

In [2]:
Dir_W_M = []
Dir_WO_M = []
Dirs = []
Label = []


In [3]:
path_with_mask = "with_mask"
path_without_mask = "without_mask"

In [4]:
for r, d, f in os.walk(path_with_mask):
    for file in f:
        Dir_W_M.append(os.path.join(r, file))
        
for r, d, f in os.walk(path_without_mask):
    for file in f:
        Dir_WO_M.append(os.path.join(r, file))

## Preprocessing Steps:

In [5]:
for file in Dir_W_M:
    img = cv2.imread(file)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    resize = cv2.resize(gray,(200,200))
    Dirs.append(resize/255)
    Label.append('with_mask')
for file in Dir_WO_M:
    img = cv2.imread(file)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    resize = cv2.resize(gray,(200,200))
    Dirs.append(resize/255)
    Label.append('without_mask')

In [6]:
data = np.array(Dirs)
label_encoder = preprocessing.LabelEncoder() 
target = label_encoder.fit_transform(Label)

In [7]:
len(data)

930

In [8]:
data[0]

array([[0.22352941, 0.18823529, 0.19607843, ..., 0.0745098 , 0.15294118,
        0.1372549 ],
       [0.20392157, 0.18823529, 0.20392157, ..., 0.18431373, 0.10588235,
        0.15294118],
       [0.18823529, 0.2       , 0.21176471, ..., 0.25490196, 0.08235294,
        0.17254902],
       ...,
       [0.25098039, 0.24705882, 0.25098039, ..., 0.05882353, 0.03921569,
        0.03921569],
       [0.24705882, 0.23921569, 0.23921569, ..., 0.05098039, 0.05490196,
        0.05490196],
       [0.23137255, 0.22352941, 0.21960784, ..., 0.0627451 , 0.05882353,
        0.05882353]])

In [12]:
data[0].shape

(200, 200)

In [9]:
new_data = []
for i in data:
    new_data.append(i.reshape(-1))
    

In [13]:
len(new_data)

930

In [10]:
print(new_data[0])

[0.22352941 0.18823529 0.19607843 ... 0.0627451  0.05882353 0.05882353]


In [11]:
new_data[0].shape

(40000,)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(new_data, target, test_size=0.20)

In [55]:
len(X_train)

744

In [56]:
len(X_test)

186

## Random Forest Classifier

In [48]:
from sklearn.metrics import confusion_matrix

In [46]:
from sklearn.ensemble import RandomForestClassifier

Random_forest = RandomForestClassifier(criterion='entropy')
Random_forest.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [47]:
Random_forest.score(X_test,y_test)

0.978494623655914

In [50]:
y_pred = Random_forest.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[130,   1],
       [  3,  52]], dtype=int64)

### Accuracy
(TP + TN) / (TP+FP+FN+TN) = (130+52)/ (186) = 0.962962

### Percision
TP/ (TP+FP) = 130/ 131 = 0.9923664122

### Recall
TP / (TP+FN) = 130 / 133 = 0.977443609

### Specificity
TN / (TN+FP) = 52 / 53 = 0.9811320755

### F1 - SCORE
2(PRECISION * RECALL)/ (PRECISION + RECALL) = 2( 0.992 0.977) / (0.992 + 0.977) = 0.9844428644

## K-Neighbors Classifier

In [25]:
from sklearn.neighbors import KNeighborsClassifier

KNN_model = KNeighborsClassifier(n_neighbors=5)

KNN_model.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [26]:
KNN_model.score(X_test,y_test)

0.8817204301075269

In [51]:
y_pred = KNN_model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[121,  10],
       [ 12,  43]], dtype=int64)

### Accuracy
(TP + TN) / (TP+FP+FN+TN) = (121 + 43)/ (186) = 0.8817204301

### Percision
TP/ (TP+FP) = 121/ 131 = 0.9236641221

### Recall
TP / (TP+FN) = 121 / 133 = 0.9097744361

### Specificity
TN / (TN+FP) = 43 / 53 = 0.8113207547

### F1 - SCORE
2(PRECISION * RECALL)/ (PRECISION + RECALL) = 2( 0.923 * 0.909) / ( 0.923 + 0.909) = 0.9159465066

##  Support Vector Machine

In [28]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [29]:
svclassifier.score(X_test,y_test)

0.9193548387096774

In [52]:
y_pred = svclassifier.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[123,   8],
       [  7,  48]], dtype=int64)

### Accuracy
(TP + TN) / (TP+FP+FN+TN) = (123 + 48)/ (186) = 0.9193548387

### Percision
TP/ (TP+FP) = 123/ 131 = 0.9389312977

### Recall
TP / (TP+FN) = 123 / 130 = 0.9461538462

### Specificity
TN / (TN+FP) = 48 / 56 = 0.8571428571

### F1 - SCORE
2(PRECISION * RECALL)/ (PRECISION + RECALL) = 2( 0.938 * 0.946) / ( 0.938 + 0.946) = 0.9419830149

## Logestic regression

In [43]:
from sklearn.linear_model import LogisticRegression
Logestic = LogisticRegression(random_state=0).fit(X_train, y_train)

C:\Users\OVIYA H\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [44]:
Logestic.score(X_test,y_test)

0.8978494623655914

In [53]:
y_pred = Logestic.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[125,   6],
       [ 13,  42]], dtype=int64)

### Accuracy
(TP + TN) / (TP+FP+FN+TN) = (125 + 42)/ (186) = 0.8978494623

### Percision
TP/ (TP+FP) = 125 / 131 = 0.9541984733

### Recall
TP / (TP+FN) = 125 / 138 = 0.9057971014

### Specificity
TN / (TN+FP) = 42 / 48 = 0.875

### F1 - SCORE
2(PRECISION * RECALL)/ (PRECISION + RECALL) = 2( 0.954 * 0.905) / ( 0.954 + 0.905) = 0.9288542227

## Real time Model Prection

In [45]:
Source = cv2.VideoCapture(0) 
  
while(True): 
    ret, frame = Source.read() 
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

    faces = face_cascade.detectMultiScale(frame)
    
    for (x,y,w,h) in faces:
        crop_img = frame[y:y+h, x:x+w]
        gray = cv2.cvtColor(crop_img,cv2.COLOR_BGR2GRAY)  
        resize = cv2.resize(gray,(200,200))

        resize = resize/255
        data = np.array(resize)
        
        data = data.reshape(-1)
        result = Logestic.predict([data])
        print(result)
        
        if result == 0:
            cv2.rectangle(frame, (x, y), (x + w, y + h),(0,255,0), 2) 
            cv2.putText(frame, 'With MASK', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,255), 2)
        else:
            cv2.rectangle(frame, (x, y), (x + w, y + h),(0,0,255), 2) 
            cv2.putText(frame, 'Without mask', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,255), 2)
            
        
    
    cv2.imshow('img',frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

  
Source.release() 
cv2.destroyAllWindows() 

[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[0]
[0]
[1]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[1]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[1]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
